In [252]:
import netCDF4 as nc
import numpy as np
import os
import re
import torch
from torch import nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader


In [80]:
# Define variable names and corresponding mean and std values
features = ['PS', 'Z3', 'U', 'V', 'T', 'lat', 'lon', 'DSE', 'RHOI', 'NETDT', 'NM', 'UTGWSPEC', 'VTGWSPEC']

directory_path = '../Demodata/Convection'
file_path_mean = '../Demodata/Convection/mean_demo_sub.npz'
file_path_std = '../Demodata/Convection/std_demo_sub.npz'

In [267]:
def load_variables(directory_path, variable_names, startfile, endfile):
    # Define the variable mapping
    variable_mapping = {
        'NM': 'NMBV'
    }

    # Dictionary to store data for each variable
    variable_data = {}

    # Pattern to match file names
    pattern = re.compile(r'^newCAM_demo_sub_\d{startfile,endfile}$')

    # Iterate over each data file in the directory
    for file_name in os.listdir(directory_path):
        # Check if the file starts with 'newCAM_demo_sub_'
        if file_name.startswith('newCAM_demo_sub_'):
            # Construct the full file path
            file_path = os.path.join(directory_path, file_name)

            # Load data from the file
            with nc.Dataset(file_path) as dataset:
                # Iterate over each variable name
                for var_name in variable_names:
                    # Check if the variable exists in the dataset
                    mapped_name = variable_mapping.get(var_name, var_name)
                    if mapped_name in dataset.variables:
                        # Read the variable data
                        var_data = dataset[mapped_name][:]

                        # Store the variable data in the dictionary
                        variable_data[var_name] = var_data

    return variable_data


def load_mean_std(file_path_mean, file_path_std, variable_names):
    
    # Load mean and standard deviation files
    mean_data = np.load(file_path_mean)
    std_data = np.load(file_path_std)

    # Define dictionaries to store mean and std for each variable
    mean_dict = {var_name: mean_data[var_name] for var_name in variable_names}
    std_dict = {var_name: std_data[var_name] for var_name in variable_names}

    return mean_dict, std_dict



def normalize_data(variable_data, mean_values, std_values):
    
    normalized_data = {}

    # Iterate over each variable in the variable data
    for var_name, var_data in variable_data.items():
        # Check if variable exists in the mean and std dictionaries
        if var_name in mean_values and var_name in std_values:
            # Extract mean and std for the variable
            mean = mean_values[var_name]
            std = std_values[var_name]

            # Perform normalization
            normalized_var_data = (var_data - mean) / std

            # Store normalized data
            normalized_data[var_name] = normalized_var_data

    return normalized_data


In [293]:
variable_data = load_variables(directory_path, features, 1, 5)
# print(f'Data variables: {variable_data.keys()}')
mean_dict, std_dict = load_mean_std(file_path_mean, file_path_std, features)
# print(f'Mean variables: {mean_dict.keys()}')
# print(f'Std variables: {std_dict.keys()}')
normalized_data = normalize_data(variable_data, mean_dict, std_dict)
# print(f'Normalised variables: {normalized_data.keys()}')


for var_name, var_data in normalized_data.items():
    # Get the shape of the variable data
    var_shape = var_data.shape if isinstance(var_data, np.ndarray) else "Not an array"
    print("Variable:", var_name, " Shape:", var_shape)

Variable: PS  Shape: (1, 4419)
Variable: Z3  Shape: (1, 93, 4419)
Variable: U  Shape: (1, 93, 4419)
Variable: V  Shape: (1, 93, 4419)
Variable: T  Shape: (1, 93, 4419)
Variable: lat  Shape: (1, 4419)
Variable: lon  Shape: (1, 4419)
Variable: DSE  Shape: (1, 93, 4419)
Variable: RHOI  Shape: (1, 94, 4419)
Variable: NETDT  Shape: (1, 93, 4419)
Variable: NM  Shape: (1, 93, 4419)
Variable: UTGWSPEC  Shape: (1, 93, 4419)
Variable: VTGWSPEC  Shape: (1, 93, 4419)


Correct NCol = 4419
ilev = 93

Points to be considered:
1. Some variables are different dimensions, varying over different levels (ilev=93/94 here)
2. These levels cause the input dimensions to become large (8 variables, each with 93 instances (i.e.varying across 93 vertical levels), and 4 variables not varying across the 93 levels.)
3. Both Input and Ouput variables have 93 levels

In [256]:
def data_loader(variable_names, normalized_data, ilev):
    # Determine the shape of the data
    Ncol = normalized_data[variable_names[1]].shape[2]
    # print(f'ilev= {ilev} and Ncol={Ncol}')

    # Initialize x_train and y_train arrays
    # Calculate dim_NN and dim_NNout
    dim_NN = int(8 * ilev + 4) # 8 variables varying over 93 levels, 4 constant variables (lat, long, PS )
    dim_NNout = int(2 * ilev) #(UTGWSPEC, VTGWSPEC)

    # Initialize x_train and y_train arrays
    x_train = np.zeros([dim_NN, Ncol])
    y_train = np.zeros([dim_NNout, Ncol])

    # print(f'Set xtrain shape{x_train.shape}')
    # print(f'Set ytrain shape{y_train.shape}')
    target_var = ['UTGWSPEC','VTGWSPEC']

    # Assign variables to x_train
    y_index = 0
    x_index = 0
    for var_name, var_data in normalized_data.items():
        var_shape = var_data.shape

        if var_name in target_var:
            # print(var_name, y_index
            y_train[y_index * ilev:(y_index + 1) * ilev, :] = var_data.reshape(ilev, Ncol)
            y_index +=1
         
        elif len(var_shape) == 2:  # For 2D variables
            #  print(var_name, x_index)
             x_train[x_index, :] = var_data
          
        elif len(var_shape) == 3:
            new_ilev = var_shape[1]
            # print(var_name, x_index)
            x_train[x_index:x_index + new_ilev, :] = var_data ### Issue here in extracting variables level-wise because of difference in levels
        x_index+=1

    return x_train, y_train


In [294]:
xtrain, ytrain = data_loader(features, normalized_data, ilev=93)
print(xtrain.shape, ytrain.shape)

(748, 4419) (186, 4419)


In [258]:
# Required for feeding the data into NN.
class myDataset(Dataset):
    def __init__(self, X, Y):
        """
        Parameters:
            X (tensor): Input data.
            Y (tensor): Output data.
        """
        self.features = torch.tensor(X, dtype=torch.float64)
        self.labels = torch.tensor(Y, dtype=torch.float64)

    def __len__(self):
        """Function that is called when you call len(dataloader)"""
        return len(self.features.T)

    def __getitem__(self, idx):
        """Function that is called when you call dataloader"""
        feature = self.features[:, idx]
        label = self.labels[:, idx]

        return feature, label


In [259]:
data = myDataset(X=xtrain, Y=ytrain)
split_data = torch.utils.data.random_split(data, [0.75, 0.25],generator=torch.Generator().manual_seed(42))

train_dataloader = DataLoader(split_data[0], batch_size=128, shuffle=True)
val_dataloader = DataLoader(split_data[1], batch_size=len(split_data[1]), shuffle=True)

In [260]:
class FullyConnected(nn.Module):
    def __init__(self):
        """Create an instance of FullyConnected NN model."""
        super(FullyConnected, self).__init__()
        ilev = 93
        hidden_layers = 8  # Number of hidden layers
        hidden_size = 500  # Number of neurons in each hidden layer

        layers = []
        input_size = 8 * ilev + 4
        for _ in range(hidden_layers):
            layers.append(nn.Linear(input_size, hidden_size, dtype=torch.float64))
            layers.append(nn.SiLU())
            input_size = hidden_size

        layers.append(nn.Linear(hidden_size, 2 * ilev, dtype=torch.float64))

        self.linear_stack = nn.Sequential(*layers)

    def forward(self, X):
        return self.linear_stack(X)


In [261]:
class EarlyStopper:
    def __init__(self, patience=1, min_delta=0):
        """Create an instance of EarlyStopper class."""
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.min_validation_loss = np.inf

    def early_stop(self, validation_loss, model=None):
        """
        Check if early stopping condition is met.

        Args:
            validation_loss (float): Loss value on the validation set.
            model (nn.Module, optional): Model to be saved if early stopping condition is met.

        Returns
        -------
            bool: True if early stopping condition is met, False otherwise.
        """
        if validation_loss < self.min_validation_loss:
            self.min_validation_loss = validation_loss
            self.counter = 0

            # Save model
            if model is not None:
                torch.save(model.state_dict(), 'conv_torch.pth')

        elif validation_loss > (self.min_validation_loss + self.min_delta):
            self.counter += 1
            if self.counter >= self.patience:
                return True
        return False


In [262]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    avg_loss = 0
    for batch, (X, Y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, Y)

        # Backpropagation
        optimizer.zero_grad(set_to_none=True)
        loss.backward()
        optimizer.step()

        avg_loss += loss.item()  # Accumulate loss as a float

    avg_loss /= len(dataloader)

    return avg_loss



def val_loop(dataloader, model, loss_fn):
    avg_loss = sum(loss_fn(model(X), Y).item() for X, Y in dataloader) / len(dataloader)
    return avg_loss

In [263]:
def train_with_early_stopping(train_dataloader, val_dataloader, model, optimizer, criterion, early_stopper, epochs=100):
    train_losses = []
    val_losses = [0]

    for epoch in range(epochs):
        if epoch % 2 == 0:
            print(f"Epoch {epoch+1}\n-------------------------------")
            print(val_losses[-1])
            print('counter=' + str(early_stopper.counter))

        train_loss = train_loop(train_dataloader, model, criterion, optimizer)
        train_losses.append(train_loss)

        val_loss = val_loop(val_dataloader, model, criterion)
        val_losses.append(val_loss)

        if early_stopper.early_stop(val_loss, model):
            print("BREAK!")
            break

    return train_losses, val_losses


In [264]:
learning_rate = 1e-5
epochs = 100

model = FullyConnected()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
criterion = nn.MSELoss()
early_stopper = EarlyStopper(patience=5, min_delta=0)

train_losses, val_losses = train_with_early_stopping(train_dataloader, val_dataloader, model, optimizer, criterion, early_stopper, epochs)


Epoch 1
-------------------------------
0
counter=0
Epoch 3
-------------------------------
0.8489934648869807
counter=0
Epoch 5
-------------------------------
0.848900956575377
counter=0
Epoch 7
-------------------------------
0.8488277427356834
counter=0
Epoch 9
-------------------------------
0.8487524616774139
counter=0
Epoch 11
-------------------------------
0.8486892791301094
counter=0
Epoch 13
-------------------------------
0.8486358109089807
counter=0
Epoch 15
-------------------------------
0.8485900223018678
counter=0
Epoch 17
-------------------------------
0.8485513591271571
counter=0
Epoch 19
-------------------------------
0.8485374125600498
counter=0
Epoch 21
-------------------------------
0.8485556882976334
counter=2
Epoch 23
-------------------------------
0.8485675317451268
counter=4
BREAK!


In [275]:

def predict(input_data, model):
    # Convert input data to tensors
    input_tensors = {key: torch.tensor(value) for key, value in input_data.items()}

    # Ensure model is in evaluation mode
    model.eval()

    # Forward pass to make predictions
    with torch.no_grad():
        predictions = model(**input_tensors)

    return predictions


In [287]:
test_data = load_variables(directory_path, features, 5,6)
mean_dict, std_dict = load_mean_std(file_path_mean, file_path_std, features)
normalized_test_data = normalize_data(test_data, mean_dict, std_dict)

In [284]:
model_path = 'conv_torch.pth'
model = FullyConnected()
model.load_state_dict(torch.load(model_path))

<All keys matched successfully>

In [291]:

x_test, y_test = data_loader(features, normalized_test_data, ilev=93)
print(x_test.shape, y_test.shape)

test_data = myDataset(X=x_test, Y=y_test)

test_loader = DataLoader(data, batch_size=len(data), shuffle=False)

(748, 4419) (186, 4419)
